<a href="https://colab.research.google.com/github/AceR00kIe/trading_view_cookie/blob/main/worker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Package

In [ ]:
is_client = False # if job == 'BTs' and part == '0' else False

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

file_path = '/content/drive/MyDrive/Co-operate_Training/'

# Install dependencies

try :
    import nest_asyncio
    import os
    from distributed import Scheduler, Client, Worker, Nanny
    from pyngrok import ngrok
    from pinggy import Tunnel
    import asyncio
    import re
except :
    !pip install dask distributed pyngrok nest_asyncio pinggy --quiet

    import nest_asyncio
    import os
    from distributed import Scheduler, Client, Worker, Nanny
    from pyngrok import ngrok
    from pinggy import Tunnel
    import asyncio
    import re

nest_asyncio.apply()

async def async_wait_for_workers(client, required_workers=2, timeout=180, poll_interval=1):
    start = asyncio.get_event_loop().time()
    while True:
        info = client.scheduler_info()
        n_workers = len(info['workers'])
        if n_workers >= required_workers:
            print(f"{n_workers} workers connected.")
            break
        if asyncio.get_event_loop().time() - start > timeout:
            raise TimeoutError(f"Timeout waiting for {required_workers} workers.")
        print(f"Waiting for workers... currently {n_workers} / {required_workers} connected.")
        await asyncio.sleep(poll_interval)


async def start_scheduler(port, public_url):
    scheduler = await Scheduler(host="0.0.0.0", port=port, dashboard_address="0.0.0.0:8787")
    print(f"Scheduler started at {public_url} | {scheduler.address}")

    # Save to Google Drive file
    scheduler_address_path = f'{file_path}colab_aws/dask_scheduler_address.txt'
    ## Ensure file is exist
    os.makedirs(os.path.dirname(scheduler_address_path), exist_ok=True)
    with open(scheduler_address_path, 'w') as f:
        f.write(public_url)
    print(f"Scheduler address saved to {scheduler_address_path}")

    return scheduler

def task() :
    pass

def close_worker_sync(dask_worker):
    asyncio.run(dask_worker.close())

async def run_client(scheduler_address, scheduler):
    # Connect client to scheduler
    client = await Client(scheduler_address, asynchronous=True)
    print(client)
    print(client.scheduler_info())
    print(f"Client connected to scheduler at {scheduler_address}")

    # Wait for more worker incoming
    print("Waiting worker")
    await async_wait_for_workers(client)#, required_workers=2)
    #await client.wait_for_workers(n_workers=2)

    # Define some Dask-distributed tasks
    def square(x):
        print(f"Worker is computing square({x})")
        return x ** 2

    # Submit tasks to cluster asynchronously
    print("Client Sending tasks")
    futures = client.map(square, range(20))

    # Gather and print results when ready
    print("Client Gathering results")
    results = await client.gather(futures, direct=False) # asynchronous=True,
    print("Results from distributed computation:", results)

    # Wait for worker task to finish (if started as asyncio task)
    #await worker_task
    #await client.run(lambda dask_worker: dask_worker.close())
    #await client.run(close_worker_sync)
    #print("Client closing worker")

    # Close client connection
    #await client.close()
    #print("Client closed")

    # Shutdown scheduler
    #await scheduler.close()
    #print("Scheduler closed")

    return results

"""
async def start_worker(scheduler_address):
    worker = await Worker(scheduler_address, nthreads=2, memory_limit="4GB")
    print(f"Worker connected to scheduler at {scheduler_address}")
    await worker.finished()
"""

async def start_pinggy(port=9001):
    process = await asyncio.create_subprocess_exec(
        "ssh",
        "-o", "StrictHostKeyChecking=no",
        "-p", "443",
        f"-R0:localhost:{port}",
        "tcp@a.pinggy.io", ## we need TCP port
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.STDOUT
    )

    while True:
        line = await process.stdout.readline()
        if not line:
            return None
        decoded = line.decode().strip()
        print(decoded)
        #match = re.findall(r'tcp://[a-zA-Z0-9\-\.]+\.free.pinggy\.', decoded)
        match = re.findall(r'tcp://[a-zA-Z0-9\-\.]+\.pinggy\.link:\d+', decoded)
        if match:
            #public_url = match.group(1)
            public_url = match[-1] ## <--- this should be tcp://.......
            print(f"Pinggy (localhost:{port}) tunnel opened at: {public_url}")
            #break
            return public_url

async def start_worker(scheduler_address, worker_address=None, worker_port=None):
    # Run the async function
    if worker_address == None and worker_port :
        ## using pinggy to get public url
        worker_address = await start_pinggy(worker_port)

    while True:
        try:
            nthreads     = 2
            memory_limit = "10GB"
            if worker_address and worker_port :
                #worker = await Worker(scheduler_address, host="0.0.0.0", contact_address=worker_address, port=worker_port) #nthreads=nthreads, memory_limit=memory_limit,
                worker = await Nanny(scheduler_address, host="0.0.0.0", contact_address=worker_address, port=worker_port, name=worker_address) #nthreads=nthreads, memory_limit=memory_limit,
                print(f"Worker binding to 0.0.0.0:{worker_port}, public address: {worker_address}")
            else :
                #worker = await Worker(scheduler_address, host="0.0.0.0") #, nthreads=nthreads, memory_limit=memory_limit
                worker = await Nanny(scheduler_address, host="0.0.0.0") #, nthreads=nthreads, memory_limit=memory_limit
            print(f"Worker connected to scheduler at {scheduler_address}")
            await worker.finished()  # blocks here until worker shuts down
            break
        except Exception as e:
            print(f"Worker failed with {e}, restarting in 5 seconds...")
            await asyncio.sleep(5)  # retry after delay


try : ## Disconnect if exist
    ngrok.disconnect(public_tunnel)
except :
    pass

if is_client : ## Client Part + Scheduler + Worker

    # === ngrok authtoken here ===
    NGROK_AUTH_TOKEN = "2z1TC2xKjmC8LYOYg77KHFZX5Cm_5NRMuVVdn7fN9cJkuCLTZ"  # Replace this with your real token
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)

    SCHEDULER_PORT = 8786 ## Dask port
    DASHBOARD_PORT = 8787

    # Open ngrok TCP tunnel to expose scheduler port publicly
    public_tunnel    = ngrok.connect(SCHEDULER_PORT, "tcp")
    dashboard_tunnel = ngrok.connect(DASHBOARD_PORT, "http")

    SCHEDULER_ADDRESS = public_tunnel.public_url
    DASHBOARD_ADDRESS = dashboard_tunnel.public_url

    print(f"Ngrok public URL (for workers and client): {SCHEDULER_ADDRESS}")
    print(f"Dashboard ngrok TCP URL: {DASHBOARD_ADDRESS}")

    async def main():
        print("Starting Scheduler")
        # Start scheduler (just like server channel for communication)
        scheduler = await start_scheduler(SCHEDULER_PORT, SCHEDULER_ADDRESS)
        print("Scheduler running")

        # wait the scheduler completed
        await asyncio.sleep(2)

        # be part of worker after client assign job
        #await start_worker(SCHEDULER_ADDRESS) ## This will blocking

        # Start worker(s) concurrently (does not block)
        print("Starting Worker")
        worker_task = asyncio.create_task(start_worker(SCHEDULER_ADDRESS))
        print("Worker running")
        # wait the scheduler completed
        await asyncio.sleep(2)

        # Run client jobs after scheduler started
        #await run_client(SCHEDULER_ADDRESS, scheduler)

        # Run client jobs (client is async)
        print("Starting Client")
        results = await run_client(SCHEDULER_ADDRESS, scheduler)

        # Keep scheduler alive so workers can connect
        await scheduler.finished()
        print("schedule finished")

        # Delete the previous scheduler address path
        if os.path.exists(scheduler_address_path):
            os.remove(scheduler_address_path)
            print(f"Deleted file: {scheduler_address_path}")
        else:
            print("File does not exist.")

        # Shutdown workers gracefully
        await client.run(lambda dask_worker: dask_worker.close())

        # Close client connection
        await client.close()

    # Run the combined scheduler + client event loop
    asyncio.get_event_loop().run_until_complete(main())

else : ## Worker Part

    """
    ## Ngrok version
    NGROK_AUTH_TOKEN = "2z68NN59CTemkEVI5hg6Qr9FQDa_251TVBwa47obznQZ22JY4"
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    WORKER_PORT      = 9001 ## Dask port
    worker_tunnel    = ngrok.connect(WORKER_PORT, "tcp")
    WORKER_ADDRESS   = worker_tunnel.public_url
    print(f"worker tunnel opened at: {WORKER_ADDRESS}")
    """
    WORKER_PORT      = 9001 ## Dask port

    # Load from Google Drive file
    scheduler_address_path = f'{file_path}colab_aws/dask_scheduler_address.txt'
    # Read the scheduler address
    if os.path.exists(scheduler_address_path):
        with open(scheduler_address_path, 'r') as f:
            SCHEDULER_ADDRESS = f.read().strip()
    else :
        print("scheduler_address_path : Not found")

    print("Scheduler address read from file:", SCHEDULER_ADDRESS)
    #SCHEDULER_ADDRESS = "tcp://0.tcp.ngrok.io:XXXXX"  # Replace with your ngrok URL

    async def main() :
        print("Starting Worker")
        #worker_task = asyncio.create_task(start_worker(SCHEDULER_ADDRESS, WORKER_ADDRESS, WORKER_PORT))
        worker_task = asyncio.create_task(start_worker(SCHEDULER_ADDRESS, worker_port=WORKER_PORT))
        print("Worker running")
        await worker_task

    # Run the worker
    asyncio.get_event_loop().run_until_complete(main())

    try : ## Disconnect if exist
        #ngrok.disconnect(worker_tunnel)
        worker_tunnel.stop()
    except :
        pass



Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 48.9 MB/s eta 0:00:00
Scheduler address read from file: tcp://8.tcp.ngrok.io:14764
Starting Worker
Worker running
Pseudo-terminal will not be allocated because stdin is not a terminal.
Allocated port 3 for remote forward to localhost:9001


INFO:distributed.worker:      Start worker at: tcp://rnvpa-35-237-96-255.a.free.pinggy.link:42593
INFO:distributed.worker:         Listening to: tcp://rnvpa-35-237-96-255.a.free.pinggy.link:42593
INFO:distributed.worker:         dashboard at: rnvpa-35-237-96-255.a.free.pinggy.link:45459
INFO:distributed.worker:Waiting to connect to: tcp://8.tcp.ngrok.io:14764
INFO:distributed.worker:-------------------------------------------------
INFO:distributed.worker:              Threads:                          2
INFO:distributed.worker:               Memory:                  12.67 GiB
INFO:distributed.worker:      Local Directory: /tmp/dask-scratch-space/worker-e7oy868r
INFO:distributed.worker:-------------------------------------------------


You are not authenticated.
Your tunnel will expire in 60 minutes. Upgrade to Pinggy Pro to get unrestricted tunnels. https://dashboard.pinggy.io
tcp://rnvpa-35-237-96-255.a.free.pinggy.link:42593
Pinggy (localhost:9001) tunnel opened at: tcp://rnvpa-35-237-96-255.a.free.pinggy.link:42593


INFO:distributed.worker:Starting Worker plugin shuffle
INFO:distributed.worker:        Registered to: tcp://8.tcp.ngrok.io:14764
INFO:distributed.worker:-------------------------------------------------
INFO:distributed.core:Starting established connection to tcp://8.tcp.ngrok.io:14764


Worker binding to 0.0.0.0:9001, public address: tcp://rnvpa-35-237-96-255.a.free.pinggy.link:42593
Worker connected to scheduler at tcp://8.tcp.ngrok.io:14764


INFO:distributed.worker:Run out-of-band function 'set'
INFO:distributed.worker:Run out-of-band function 'set'
